In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dmt.d_store_performance_tier
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS 
WITH total_rev AS 
(
  SELECT 
    calday,
    store_id,
    business_unit,
    SUM(IF(business_unit = '1500', revenue.rev_amt, revenue.rev_amt_pos)) AS revenue
  FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily a
  INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b USING (store_id)
  INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c USING (product_id)
  WHERE calday BETWEEN add_months(trunc(current_date(), 'MONTH'), -6) AND
   date_sub(trunc(current_date(), 'MONTH'), 1)
  AND business_unit IN ('1500', '2000')
  AND mch2_id IN ('101', '102', '201', '202', '203')
  GROUP BY ALL 
  HAVING SUM(IF(business_unit = '1500', revenue.rev_amt, revenue.rev_amt_pos))   <> 0
)
, rev_perday_bystore AS 
(
  SELECT 
    store_id,
    business_unit,
    SUM(revenue) AS rev, 
    COUNT(calday) AS cnt_day,
    TRY_DIVIDE(SUM(revenue), COUNT(calday)) AS rev_perday
  FROM total_rev
  GROUP BY 1, 2
)
, store_performance_tier AS 
(
  SELECT 
    a.*,
    CASE 
      WHEN business_unit = '1500' THEN 'NA'
      WHEN rev_perday > 25 * 1E6 THEN 'N6'
      WHEN rev_perday < 25 * 1E6 AND rev_perday >= 18.1 * 1E6 THEN 'N5'
      WHEN rev_perday < 18 * 1E6 AND rev_perday >= 15.1 * 1E6 THEN 'N4'
      WHEN rev_perday < 15 * 1E6 AND rev_perday >= 12.1 * 1E6 THEN 'N3'
      WHEN rev_perday < 12 * 1E6 AND rev_perday >= 8.1 * 1E6 THEN 'N2'
      ELSE 'N1'
    END AS store_group
  FROM rev_perday_bystore a
)

SELECT 
    *,
    CASE 
      WHEN store_group = 'N6' THEN 'a. > 25 mil.'
      WHEN store_group = 'N5' THEN 'b. 18 - 25 mil.'
      WHEN store_group = 'N4' THEN 'c. 15 - 18 mil.'
      WHEN store_group = 'N3' THEN 'd. 12 - 15 mil.'
      WHEN store_group = 'N2' THEN 'e. 8 - 12 mil.'
      WHEN store_group = 'N1' THEN 'f. < 8 mil.'
      WHEN store_group = 'NA' THEN 'NA'
    END AS performance_tier
FROM store_performance_tier """)